In [ ]:
import sklearn
import numpy as np
import pandas

import sklearn
from sklearn import tree
from sklearn import ensemble

import skimage.data
import skimage.transform
from skimage import data, io, filters

In [ ]:
import math

width = 240
height = 160

# Import Labels
labels = []
imagePaths = []
lastTrackingAngle = 0
for i in range(20,26):
    fname = "./CarDataNew/AccelerationData/capture"+str(i)+"/accelClass.txt"
    base = "./CarDataNew/AccelerationData/capture"+str(i) + "/"
    with open(fname) as f:
        content = f.readlines()
        for line in content:
            line = line.split(",")
            speedString = (line[1].split(" "))[2]
            speed = float(speedString)
            trackingAngleString = (line[8].split(" "))[2]
            trackingAngle = float(trackingAngleString)
            deltaTrackingAngle = trackingAngle - lastTrackingAngle
            lastTrackingAngle = trackingAngle
            filePath = (line[10])
            if speed > 10 and abs(deltaTrackingAngle) > 1 and abs(deltaTrackingAngle) < 4:
                labels.append(deltaTrackingAngle)
                #print speed
                #print trackingAngle
                picturePath = base+filePath
                #print picturePath
                imagePaths.append(picturePath)

trainingIndex = int(len(imagePaths)/1.3)

In [ ]:
images = []
for i in range(len(imagePaths)):
    loadPath = imagePaths[i]
    images.append(skimage.data.imread(loadPath))

In [ ]:
# Resize images
images = [skimage.transform.resize(image, (height, width))
                for image in images]

In [ ]:

images = np.asarray(images)
imagesFlat = images.reshape(len(images),width*height*3)

In [ ]:
labels_a = np.array(labels)
images_a = np.array(imagesFlat)
print("labels: ", labels_a.shape, "\nimages: ", imagesFlat.shape)

In [ ]:
RegressorClass = labels
clf = ensemble.RandomForestRegressor(n_jobs=-1)
for round in range(0,1):
    print("training start")
    clf = clf.fit(imagesFlat[:trainingIndex], labels[:trainingIndex])
    print("training end")

In [ ]:
#Calculate MSE for testing        
print(clf.score(imagesFlat[trainingIndex:], labels[trainingIndex:]))

In [ ]:
totalSquaredError = 0
for i in range(trainingIndex, len(images)):
    imageToPredict = imagesFlat[i]
    imageToPredict= imageToPredict.reshape(1, -1)
    result = clf.predict(imageToPredict)
    guess = result
    actual = labels[i]
    print("Guess: " + str(guess) + "   Actual: " + str(actual))
    squaredError = abs(actual-guess)*abs(actual-guess)
    totalSquaredError += squaredError
averageSquaredError = math.sqrt(totalSquaredError/(len(images)-trainingIndex))
print(str(averageSquaredError))